# Contents
## Installation
### Set up channels
### Create an environment and install the packages
## Alignment
### HISAT 
### Samtools
## ChIP-Seq
### MACS
### Bedtools
## RNA-Seq
### HTSeq
### featureCounts
### DESeq


# Installation

Before running any programs, we'll make sure that each software is installed correctly. This tutorial uses Bioconda (https://bioconda.github.io/). Bioconda is a channel for the conda package manager specializing in bioinformatics software. The available packages are listed here: https://bioconda.github.io/recipes.html#recipes.

## Set up channels

You will need to add the bioconda channel as well as the other channels bioconda depends on. It is important to add them in this order so that the priority is set correctly (that is, bioconda is highest priority).

The conda-forge channel contains many general-purpose packages not already found in the defaults channel. The r channel is only included due to backward compatibility. It is not mandatory, but without the r channel packages compiled against R 3.3.1 might not work.

This tutorial uses cells written in python and unix to perform its analyses. Lines that are written in unix are prefixed by an exclamation point. 

Select the following cell and run it. To run a cell, select the cell, click "Cell" the upper taskbar, and select "Run Cells". Or click the cell and press shift + enter.

Alternatively, the contents of any cell may be copy+pasted into the terminal emulator to run.

In [ ]:
!conda config --add channels defaults
!conda config --add channels conda-forge
!conda config --add channels bioconda

## Create an environment and install the packages

In this tutorial we will create an environment named "tutorial" and install the packages in there. Environments offer ways of installing packages in specific environments so they can be managed and run for different specifications. You can create, export, list, remove and update environments that have different versions of Python and/or packages installed in them. Switching or moving between environments is called activating the environment. You can also share an environment file.

This command will create an environment "tutorial" in which to install the packages used in this tutorial.

Run the following commands to create the environment. When it asks if you want to proceed, type "y" and press enter.

In [ ]:
!conda create -n tutorial hisat2 multiqc macs2 salmon bioconductor-deseq matplotlib ggplot pybedtools samtools bioconductor-rsamtools bedtools htseq subread bioconductor-rsubread

Then activate the environment with the following command.

In [ ]:
# For Mac and Linux

!source activate tutorial

# For Windows

!activate tutorial

# Alignment
## HISAT (Hierarchical Indexing for Spliced Alignment of Transcripts)

In this tutorial, we'll use Hisat to align the sample reads to a reference genome. Hisat automatically downloads and preprocesses the reads so they're ready to be aligned. Hisat (hierarchical indexing for spliced alignment of transcripts) is a highly efficient system for aligning reads from RNA sequencing experiments. HISAT uses an indexing scheme based on the Burrows-Wheeler transform and the Ferragina-Manzini (FM) index, employing two types of indexes for alignment: a whole-genome FM index to anchor each alignment and numerous local FM indexes for very rapid extensions of these alignments. HISAT’s hierarchical index for the human genome contains 48,000 local FM indexes, each representing a genomic region of ~64,000 bp.

In [ ]:
from pandas import read_csv

RNASeqSRARunTableFile='../data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSam = "test/" + RNASeqoutrun + ".sam"
RNASeqoutputAlignmentSummary = "test/" + RNASeqoutrun + ".txt"
RNASeqoutputMetrics = "test/" + RNASeqoutrun + ".metrics"
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

ChIPSeqSRARunTableFile='../data/ChIPSeqSRA.tsv'
ChIPSeqSRATable = read_csv(ChIPSeqSRARunTableFile, delimiter='\t')
ChIPSeqoutrun = (ChIPSeqSRATable["Run"]).astype(list)
ChIPSeqoutputSam = "test/" + ChIPSeqoutrun + ".sam"
ChIPSeqoutputAlignmentSummary = "test/" + ChIPSeqoutrun + ".txt"
ChIPSeqoutputMetrics = "test/" + ChIPSeqoutrun + ".metrics"
ChIPSeqoutputSortBam = "test/" + ChIPSeqoutrun + ".sorted.bam"

In [ ]:
Then run the following command to create the yeast index.

In [ ]:
!bash ../scripts/make_yeast_index.sh

In [ ]:
Align the RNA-Seq samples using Hisat. 

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !hisat2 -x ../yeast_index/genome --sra-acc $run --new-summary --summary-file ../$summary --met-file ../$metrics -S ../$sam

## Samtools 

We'll use samtools to sort the output files and convert them to bam files.

Sort the output files and convert them to bam files.

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !samtools view -bSF4 ../$sam | samtools sort -o ../$bam
    

In [ ]:
Do the same thing for ChIP-Seq samples.

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    index = "yeast_index/genome"
    !hisat2 -x $index --sra-acc $run --new-summary --summary-file ../../../$summary --met-file ../../../$metrics -S ../../../$sam

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    !samtools view -bSF4 ../$sam | samtools sort -o ../$bam

# ChIP-Seq
## MACS (Model-based Analysis for ChIP-Seq)

Peak-calling is one of the main steps scientists use in determining the locations where protein is bound in DNA. Peak detection software, such as MACS (Model-Based Analysis for ChIP-Seq), call peaks using the aligned sequecnes as input and returns precise locations of predicted peaks as output. In this tutorial, we'll use MACS.

More information about MACS: http://liulab.dfci.harvard.edu/MACS/Download.html

In [ ]:
ChIPSeqControl = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] == "Untreated"]
ChIPSeqTreatment = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] != "Untreated"]

In [ ]:
for index, individual in enumerate(ChIPSeqControl):
    input = individual
    immunoprecipitate = ChIPSeqTreatment[index]
    sample = ChIPSeqRun[index]
    !macs2 callpeak -c $input -t $immunoprecipitate -n $sample --outdir $sample

For an in-depth discussion of what MACS2 does: https://github.com/taoliu/MACS/wiki/Advanced:-Call-peaks-using-MACS2-subcommands

## Bedtools

In this tutorial, we'll use bedtools to extract the intersecting regions of the MACS output between the experimental conditions.

First we'll sort the output. The following line uses the `sort` command to sort the MACS output.

In [ ]:
!sort -k 1,1 -k2,2n input1 > output1
!sort -k 1,1 -k2,2n input2 > output2

Then we'll use pybedtools to truncate the output files to chromosome so no ends are out of bounds. 

In [ ]:
from pybedtools import BedTool

BedTool(output1).truncate_to_chrom('sacCer3').saveas(output_trunc1)
BedTool(output2).truncate_to_chrom('sacCer3').saveas(output_trunc2)

Then we'll find the intersecting regions between the experimental conditions and the control conditions. 

In [ ]:
!bedtools intersect -a output_trunc1 -b output_trunc2 -u > intersect12

# RNA-Seq
## HTSeq (High-through sequencing)

HTSeq is a Python library to facilitate the rapid development of RNA-Seq analysis. HTSeq offers parsers for many common data formats in HTS projects, as well as classes to represent data, such as genomic coordinates, sequences, sequencing reads, alignments, gene model information and variant calls, and provides data structures that allow for querying via genomic coordinates. In this tutorial we will use htseq-count, a tool developed with HTSeq that preprocesses RNA-Seq data for differential expression analysis by counting the overlap of reads with genes.

In [ ]:
from pandas import read_csv

gtf = "../data/Saccharomyces_cerevisiae.R64-1-1.84.gtf"

RNASeqSRARunTableFile='../data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    output = individual + ".genecount.txt"
    !htseq-count -m intersection-nonempty -s no -f bam $input $gtf > $output

## featureCounts

featureCounts is a highly efficient general-purpose read summarization program that counts mapped reads for genomic features such as genes, exons, promoter, gene bodies, genomic bins and chromosomal locations. It can be used to count both RNA-seq and genomic DNA-seq reads. It is available in the SourceForge Subread package or the Bioconductor Rsubread package.

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    !featureCounts -s -a $gtf -o ../test/featureCounts $input

## DESeq (Differential Expression Sequencing)

Estimate variance-mean dependence in count data from high-throughput sequencing assays and test for differential expression based on a model using the negative binomial distribution

In [ ]:
!Rscript ../scripts/runDeseq.R

# Salmon

Salmon is a tool for quantifying the expression of transcripts using RNA-seq data. Salmon uses new algorithms (specifically, coupling the concept of quasi-mapping with a two-phase inference procedure) to provide accurate expression estimates very quickly (i.e. wicked-fast) and while using little memory. Salmon performs its inference using an expressive and realistic model of RNA-seq data that takes into account experimental attributes and biases commonly observed in real RNA-seq data.

In [ ]:
!wget -O ../data/transcript.fa.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_rna.fna.gz

In [ ]:
Then unzip the transcript.fa.gz file.

In [ ]:
!gunzip ../data/transcript.fa.gz

In [ ]:
Finally, create the salmon index and run salmon on it.

In [ ]:
salmon_index = "../test/yeast-salmon.idx"

!salmon index -t ../data/transcript.fa -i $salmon_index --type quasi -k 31


In [ ]:
from pandas import read_csv

RNASeqSRARunTableFile='../data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"])
RNASeqBam = "test/" + RNASeqoutrun + ".sorted.bam"

for index, individual in enumerate(RNASeqBam):
    !salmon quant --targets $salmon_index -l A --output ../test -a ../$individual